# Handschriftenerkennung mit PyTorch

Zum Reinkommen und Einüben des PyTorch Frameworks habe ich die erste Programmieraufgabe, die wir noch from the scratch mit Numpy programmiert haben, mal in PyTorch umgesetzt. Am besten lernt man ein neues Framework meiner Meinung nach kennen, wenn man einfache, bekannte Ideen umsetzt. Dabei bin ich in seeeehr viele Probleme reingerannt und möchte euch mit diesem Notebook ein wenig mitnehmen, damit ihr von meinen Erfahrungen profitieren könnt. Allerdings muss ich euch ja nicht sagen, dass selbst ausprobieren mehr hilft. Daher kommt am Ende eine kleine freiwillige Aufgabe, an der ich selbst eine halbe Ewigkeit saß (die jetzt aber total trivial wirkt).

Die meisten, wirklich allermeisten Probleme hatte ich, weil ich die Datentypen nicht richtig angeschaut habe. Wie die Daten strukturiert und gespeichert sind ist echt wichtig und ich gehe Stück für Stück mit euch da durch.

## Daten, Daten, Daten

### Daten importieren

Zunächst den ganzen Spaß importieren und die Daten herunterladen. Falls ihr die schon einmal heruntergeladen habt, könnt ihr das Argument `download=False` so stehen lassen, andernfalls muss das zu `True` geändert werden.

In [ ]:
%matplaotlib inline

In [ ]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import torchvision
import sys

In [ ]:
def load_mnist_data(train):
    from torchvision import datasets, transforms
    from torch.utils.data import DataLoader
    
    transform = transforms.Compose([ transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.5,], std=[0.5,]) 
                                   ])
    dataset = datasets.MNIST('./MNIST/', train=train, transform=transform,  download=False)
    return dataset

train_dataset = load_mnist_data(train=True)
test_dataset = load_mnist_data(train=False)

Mit dem `transform` Parameter können wir die Daten so vorbereiten, wie wir sie haben wollen. Wir möchten sie wie in der Aufgabe als Werte zwischen -1 und 1 haben. Problem: Sie kommen als Werte zwischen 0 und 1. Und zwar auch nicht als Tensor, sondern als `PIL`-Objekt, ein Datentyp für Bilder. 

Der `transform.Compose` Aufruf schaltet mehrere Transformationen hintereinander, die wir dann übergeben können. Zunächst wollen wir aus dem `PIL`-Objekt einen Tensor mit `transforms.ToTensor()` bekommen. Dann möchten wir die Daten shiften. Die Funktion `transforms.Normalize` verschiebt die Daten so, dass sie Mittelwert 0 und Varianz 1 hat. Genauer:
Für Eingangsdaten $x \in [0, 1]$ berechnet die Funktion 

$$ x^* = \frac{x - \text{mean}}{\text{std}} $$

Wenn das aber die Berechnung ist, sehen wir leicht, dass wir die Daten, die zwischen 0 und 1 liegen, mit `mean=0.5` und `std=0.5` auf -1 bis 1 shiften können.

Die Daten werden als Liste `[0.5,]` übergeben, die die Werte für jeden Channel einzeln beinhaltet. Da wir nur einen Channel haben, brauchen wir nur ein Element.


### Daten genaaaaaau anschauen

Ich kann's nicht oft genug sagen: Schaut euch die Daten genau an. Wie sind sie strukturiert? Dazu geben wir einige Dinge aus:

In [ ]:
print("a:", type(train_dataset))
print("b:", type(train_dataset[0]))
print("c:", len(train_dataset[0]))
print("d:", type(train_dataset[0][0]))
print("e:", type(train_dataset[0][1]))

`train_dataset` ist ein Objekt vom Typ `torchvision.datasets.mnist.MNIST` (a). Das ist ein für diese speziellen Daten eigens programmierter Datentyp. Für uns ist nur wichtig, dass dieser iterable ist, wir können also über diese eine `for`-Schleifen laufen lassen und auf die Daten mit den eckigen Klammern `[]` zugreifen.

Tun wir dies, bekommen wir in der ersten Dimension Tupel (b) der Länge 2 (c). Die erste Komponente dieses Tupels ist ein `torch.Tensor` (d), die zweite ist ein Integer (e).

Schauen wir uns das erste Tupel mal an:

In [ ]:
print(train_dataset[0])

Dieses Tupel besteht also aus dem Signal, gespeichert als Tensor, und dem Label. Der Tensor hat folgende Größe:

In [ ]:
print(train_dataset[0][0].shape)

Nun haben wir eine gute Vorstellung über unsere Daten. 

## Parameter definieren und DataLoader instantiieren

Der DataLoader gibt uns jeweils immer einen Batch an Daten zurück, wenn wir über diesen iterieren. Das vereinfacht unseren Code. Da wir dafür schon eine unsere Netzwerkkonfigurationen benötigen (nämlich die batch_size), definieren wir hier nun gleich alle davon auf einmal. Hier wird unser Maschinenraum sein, welchen wir zum Ausprobieren anpassen können.

Damit erstellen wir uns dann unsere DataLoader...

In [ ]:
batch_size = 64
hidden_size = 10
learning_rate = 0.005
label_smoothing = 0.0
device = "cuda" if torch.cuda.is_available() else "cpu"

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset))

...welcher uns immer einen Batch lädt.

In [ ]:
for X, y in train_dataloader:
    print(X.shape)
    print(y.shape)
    break

## Neural Network definieren und trainieren


Wir bauen uns ein NN mit einem hidden layer mit 10 Logits, da wir 10 Klassen haben. Wir verwenden den Cross Entropy Loss. Hier wieder ein Punkt zu den Daten: Der Cross Entropy Loss erhält ungleiche Daten. Die targets (echte Labels) des NN sind *keine* codierten Vektoren (One-Hot-Vektoren) sondern einfache Zahlen von 0-9. Die prediction hingegen (die Ausgabe des Netzwerks) ist ein 10-elementiger Vektor mit den Logits. Die Funktion nn.CrossEntropyLoss() nimmt diese so an und berechnet auch selbstständig den Softmax. Deshalb hat unser NN keine Softmax eingebaut, sondern berechnet nur die Logits. Siehe dazu die Vorlesungsvideos, in der Prof. Frahling diesen Punkt genauer erklärt hat.

In [ ]:
# Definiere das kleine Model mit nur einem hidden layer ohne softmax
class MyNN(nn.Module):
    def __init__(self):
        super(MyNN, self).__init__()
        self.flatten = nn.Flatten(start_dim=-3, end_dim=-1)
        self.linear_stack = nn.Sequential(
            nn.Linear(28*28, hidden_size, bias=False),
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.linear_stack(x)

# Der train_dataloader gibt uns jeweils einen batch zurück. Hier sehen wir die shapes:
for X, y in train_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break
    

# Wir instantiieren das Model und weisen es ggf. der GPU zu.
model = MyNN().to(device)

# Unser CrossEntropyLoss, der Logits annimmt und den Softmax eingebaut hat
loss_fn = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

# Optimizer & SummaryWriter. 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
writer = SummaryWriter(f"zahlen_erkennen_runs/layer-1_size_{hidden_size}_bs_{batch_size}_lr_{learning_rate}_ls_{label_smoothing}")

##### Starte Training ######

"""
Zur Epochenzahl: Das haben viele von euch in Uebung 9 - zumindest in meinen Gruppen - missverstanden: Die Epochenzahl ist nicht dazu
gedacht, so viele Epochen tatsächlich durchzulaufen. Das schafft keine Hardware in angemessener Zeit.
Es soll das Training am laufen halten, damit ihr im Tensorboard die Performance live mitverfolgen
und erst bei Bedarf das Training beenden können sollt. Das Tensorboard aktualisiert on the fly. Lasst
deshalb den Wert so hoch und wenn ihr genug gesehen habt, stoppt diese Zelle manuell.
"""
epochs = 30000

# Anzahl von einzelnen Inputs (Bildern)
size = len(train_dataloader.dataset)

# Steps = Anzahl bisheriger Batches, nicht Signale
steps = 0
train_loss = 0

print(f"Working on {device}")
# Gehen wir nun das Training durch. Anders als in der Uebung 9 bspw. habe ich das alles hier
# reingeschrieben (also keine Aufgaben outgesourced in Funktionen)
for t in range(epochs):
    print(f"\nEpoche {t+1}\n-------------------------------------")
    
    # Starte Trainingsmodus des Models
    model.train()
    
    # Anzahl korrekt berechneter Labels (für die Accuracy)
    correct = 0
    
    # Gehe nun die Batches durch. Am Ende berechnen wir für jeden 10. Batch die Performance (Loss, Accuracy)
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
        
        # Das Model ist zwar nur für einzelne Signale definiert, nimmt aber auch ganze Batches auf einmal.
        # Übrigens: Wer sich wundert, dass hier ein Objekt wie eine Funktion aufgerufen wird, siehe
        # https://youtu.be/ytOytMdqD7Y für eine Erklärung
        pred = model(X)
        
        # Achtung: Hier passiert das, wovon ich oben geredet habe. Die predictions haben
        # eine Shape von (batch_size, 10), die targets y haben shape (batch_size). Die
        # Funktion CrossEntropyLoss() kann damit umgehen.
        loss = loss_fn(pred, y)
        
        # Setze Gradienten auf 0 (damit sie sich nicht aufsummieren)
        optimizer.zero_grad()
        
        # Berechne alle Gradienten
        loss.backward()
        
        # Update die Variablen mit den Gradienten
        optimizer.step()
        
        # Für jedes 10. Batch berechnen wir ...
        if batch % 10 == 0:
            
            # ...den Loss und geben sie mit der aktuellen Anzahl an berechneten Daten aus.
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
            writer.add_scalar('Loss/train', loss, steps+batch)
            
        # Wir berechnen, wie viele der Daten richtig vorausgesagt wurden. Die predictions sind
        # weiterhin eine (batch_size, 10)-geshapte Liste. pred.argmax(1) gibt den Index des größten Eintrags
        # (= der größten Wahrscheinlichkeit) für die erste Dimension (= horizontale Achse, als Matrix betrachtet)
        # aus. Das wird für alle Einträge gemacht und dies gibt einen Vektor der Länge batch_size mit Einträgen
        # True bzw. False aus, wobei der Wert jeweils dafür steht, ob die maximale Wahrscheinlichkeit dem richtigen
        # Index zugewiesen wurde. type(..) rechnet sie zu floats um (True=1.0, False=0.0) und sum() und item() geben
        # die Summe als built-in Datentyp float aus.
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        train_loss+=loss
    
    # Nun haben wir eine Epoche durchgerechnet und mitteln die Werte. Der train_loss wird
    # über die Anzahl der batches (len(train_dataloader)) gemittelt, der wurde für jeden
    # Batch aufsummiert. correct, also die Anzahl richtig vorhergesagter Signale, haben 
    # wir für jedes einzelne Signal berechnet, teilen also durch die Anzahl der Daten 
    # insgesamt (durch len(train_dataloader.dataset))
    train_loss = train_loss / len(train_dataloader) 
    correct = correct / len(train_dataloader.dataset)
    
    # Anzahl aller berechneten batches
    steps += batch
    
    writer.add_scalar('Accuracy/train', correct*100.0, steps)
    
    # Wir berechnen die Accuracy für den train loss in jeder epoch ein mal
    print(f"Train Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n") 

    
    #... und das gleiche noch mal in jeder Epoche einmal für die Testdaten. Vorher setzen wir
    # das Model auf den evaluation mode. Die "with torch.no_grad():"-Umgebung verhindert eine
    # teure Gradientenberechnung, die wir eh nicht brauchen.
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # summiert richtige vorhersagen auf
    test_loss = test_loss / len(test_dataloader) # durchschnittlicher loss auf den batches
    correct = correct / len(test_dataloader.dataset) # zwischen 0 und 1, accuracy
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")            
    writer.add_scalar('Loss/test', test_loss, steps)
    writer.add_scalar('Accuracy/test', 100.0*correct, steps)

writer.close()
    

Dieser Code schreibt auch Tensorboard-Daten. Schaut sie euch mal an! Geht dazu mit der Anaconda Shell in den Ordner `runs` (mit dem Befehl `cd`) und führt `tensorboard --logdir=.` aus.

## Unsere Variablen

Wir haben als Variable nur diese eine Matrix ohne Bias. Besonders interessant finde ich, wie diese nun aussehen. Da die erste Matrix direkt in die Logits rechnet, muss diese, damit der erste Logit bei einer 1 besonders hoch ist, genau an den Pixeln einen hohen Wert haben, damit sie sich mit der 1 zu einem hohen Skalarprodukt berechnet. Wir können jetzt diese Matrix mal als Bild interpretieren und schauen, wie sie aussehen.

In [ ]:
def show_data_as_images(data,labels=None):
    """Diese Funktion kann genutzt werden, um Daten als Bild darzustellen."""
    if len(data.shape)==3:
        #Several images - output them as an array
        cols = int(np.sqrt(data.shape[0]))
        rows = (data.shape[0]+cols-1)//cols
        figsize = 15
        if cols>5:
            figsize=3*cols
        fig = plt.figure(figsize=(figsize, figsize))  # width, height in inches
        if rows>5:
            fig.subplots_adjust(top = 1.0)
        for i,image in enumerate(data):
            sub = fig.add_subplot(cols, rows, i + 1)
            if labels is not None:
                sub.title.set_text('Label '+str(labels[i]))
            sub.imshow(image,vmin=-1, vmax=1, cmap='Greys')
        #plt.imshow(fig)
        plt.show()
    else:
        if labels is not None:
            print("Label:",labels)
        plt.imshow(data,vmin=-1, vmax=1, cmap='Greys')
        plt.show()


In [ ]:
for p in model.parameters():
    show_data_as_images(p.cpu().detach().view(-1,28,28))

Wenn ihr das lange genug laufen lassen habt, seht ihr jetzt Bilder, die wie eine Mondlandschaft mit Kratern aussieht. Legt man ein Objekt - eine Zahl - oben drüber, ist es genau dann besonders hoch, wenn diese Zahl gut in diesen Krater 'reinfällt'. Das find ich echt am interessantesten.

## Now it's your turn

Ich habe dieses Notebook geschrieben, um an einem einfachen Beispiel, das wir bereits kennen, PyTorch besser zu verstehen. Mein Appell ist, dass ihr dieses Notebook genau durchgeht und jeden einzelnen Schritt verstehen sollt, um dann kompliziertere Netzwerke verstehen/selbst bauen zu können. Spielt gerne etwas rum! Falls ihr was nicht versteht, schreibt mir. 

Als kleine Anregung: Versucht mal, den Loss zu ändern. Zum Beispiel zum mean squared error ( nn.MSELoss() ). Dieser hat keinen Softmax eingebaut! Überlegt, wo ihr diese einbauen müsst & vorallem, wie ihr die Labels transfomieren müsst, damit diese für den Loss die richtige Form haben. Tipp: Sucht nach One-Hot.

Eine Mailadresse auf der ihr mich erreichen könnt ist auf meinem GitHub Profil angegeben.

Viel Spaß!
Juan